In [14]:
import glob
import json
import csv
import pandas as pd
from pathlib import Path
import shutil
import cv2

In [ ]:
## put tdb_m and tdb_s 1-4 folder into dataset/raw folder

In [11]:
all_json = glob.glob("../dataset/raw/**/*.json", recursive=True)
sub_folder = [a.split('\\')[1] for a in all_json]
# print(all_json)
print(sub_folder)

['OK_TBS_S_2', 'OK_TBS_S_2', 'OK_TBS_S_2', 'OK_TBS_S_3', 'OK_TBS_S_3', 'OK_TBS_S_4', 'OK_TBS_S_4', 'OK_TBS_S_4', 'OK_TBS_S_4', 'OK_TDB-S_1', 'OK_TDB-S_1', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M', 'TDB_M']


In [13]:
files_grabbed = [glob.glob(e, recursive=True) for e in ['../dataset/raw/**/*.png', '../dataset/raw/**/*.jpg', '../dataset/raw/**/*jpeg']]
flat_list = [item for sublist in files_grabbed for item in sublist]
print(flat_list)

['../dataset/raw\\TDB_M\\00001.png', '../dataset/raw\\OK TBS_S 2\\00016.jpg', '../dataset/raw\\OK TBS_S 2\\00017.jpg', '../dataset/raw\\OK TBS_S 2\\00018.jpg', '../dataset/raw\\OK TBS_S 2\\00019.jpg', '../dataset/raw\\OK TBS_S 2\\00020.jpg', '../dataset/raw\\OK TBS_S 2\\00021.jpg', '../dataset/raw\\OK TBS_S 2\\00022.jpg', '../dataset/raw\\OK TBS_S 2\\00023.jpg', '../dataset/raw\\OK TBS_S 2\\00024.jpg', '../dataset/raw\\OK TBS_S 2\\00025.jpg', '../dataset/raw\\OK TBS_S 2\\00026.jpg', '../dataset/raw\\OK TBS_S 2\\00027.jpg', '../dataset/raw\\OK TBS_S 2\\00028.jpg', '../dataset/raw\\OK TBS_S 2\\00029.jpg', '../dataset/raw\\OK TBS_S 2\\00030.jpg', '../dataset/raw\\OK TBS_S 2\\00031.jpg', '../dataset/raw\\OK TBS_S 2\\00032.jpg', '../dataset/raw\\OK TBS_S 2\\00033.jpg', '../dataset/raw\\OK TBS_S 2\\00034.jpg', '../dataset/raw\\OK TBS_S 2\\00035.jpg', '../dataset/raw\\OK TBS_S 2\\00036.jpg', '../dataset/raw\\OK TBS_S 2\\00037.jpg', '../dataset/raw\\OK TBS_S 2\\00038.jpg', '../dataset/raw\\OK 

In [20]:
## Copy all image to single folder with new name => sub_folder + original filename and convert to jpg
master_path ="../dataset/processed/master_dataset/" 
path = Path(master_path)
path.mkdir(parents=True, exist_ok=True)
files_grabbed = [glob.glob(e, recursive=True) for e in ['../dataset/raw/**/*.png', '../dataset/raw/**/*.jpg', '../dataset/raw/**/*jpeg']]
flat_list = [item for sublist in files_grabbed for item in sublist]

def copy_all(input_list, output_path):
    for fn in input_list:
        split_str = fn.split("\\")
        sub_folder = split_str[1].replace(" ","_")
        real_fn = split_str[-1].split(".")[0]
        img = cv2.imread(fn)
        cv2.imwrite(output_path+sub_folder+real_fn+".jpg", img)

copy_all(flat_list,master_path)
print("All images copied and coverted to jpg")

All images copied and coverted to jpg


In [111]:
## Extract Json to CSV

fields = ["filename", "class", "fill","x","y","width","height"]
classes = ["glass","cup","plate"]

all_json = glob.glob("../dataset/raw/**/*.json", recursive=True)
print(all_json)

def json_to_csv(split=False):
    df = []
    output_name = "../dataset/processed/csv_converted.csv"
    for fn in sorted(all_json):
        sub_folder = fn.split('\\')[1].replace(" ","_")
        f = open(fn)
        json_obj = json.load(f)
        for key in json_obj:
            real_fn = json_obj[key]["filename"].split(".")[0]
            if "jpg" in real_fn:
                real_fn = real_fn[:-3]
            if "OK_TDB-S_1" in sub_folder and "Unknown" not in real_fn:
                real_fn =  "Unknown-"+real_fn[2:]
            dict_row = {"filename":sub_folder+real_fn+".jpg",
                        "class":"null",
                        "fill":"null",
                        "x":"null",
                        "y":"null",
                        "width":"null",
                        "height":"null"}

            if len(json_obj[key]["regions"]) == 0:
                df.append(dict_row)

            else:
                temp_c = []
                temp_fill = []
                temp_x =[]
                temp_y = []
                temp_width= []
                temp_height = []
                for r in json_obj[key]["regions"]:
                    if "Type" in json_obj[key]["regions"][r]["region_attributes"]:
                        c = json_obj[key]["regions"][r]["region_attributes"]["Type"].lower().strip()
                        if "Fill" in json_obj[key]["regions"][r]["region_attributes"]:
                            fill = json_obj[key]["regions"][r]["region_attributes"]["Fill"].lower().strip()
                            if fill in classes:
                                fill = 30
                    elif "type" in json_obj[key]["regions"][r]["region_attributes"]:
                        c = json_obj[key]["regions"][r]["region_attributes"]["type"].lower().strip()
                        if "fill" in json_obj[key]["regions"][r]["region_attributes"]:
                            fill = json_obj[key]["regions"][r]["region_attributes"]["fill"].lower().strip()
                            if fill in classes:
                                fill = 30
                                
                    temp_c.append(c)
                    temp_fill.append(fill)
                    
                    sa = json_obj[key]["regions"][r]["shape_attributes"]
                    x = sa["x"]
                    y = sa["y"]
                    width = sa["width"]
                    height = sa["height"]
                    
                    temp_x.append(x)
                    temp_y.append(y)
                    temp_width.append(width)
                    temp_height.append(height)
                    
                    
                if (split):
                    dict_row["class"] = temp_c
                    dict_row["fill"] = temp_fill
                    dict_row["x"] = temp_x
                    dict_row["y"] = temp_y
                    dict_row["width"] = temp_width
                    dict_row["height"] = temp_height
                else:
                    dict_row["class"] = ";".join(temp_c)
                    dict_row["fill"] = ";".join(map(str, temp_fill))
                    dict_row["x"] = ";".join(map(str, temp_x))
                    dict_row["y"] = ";".join(map(str, temp_y))
                    dict_row["width"] = ";".join(map(str, temp_width))
                    dict_row["height"] = ";".join(map(str, temp_height))
                
                df.append(dict_row)
    
    df = pd.DataFrame(df)
    df = df[fields]
    df = df.drop_duplicates(subset=['filename'], keep='last')
    print(df)
    
    if (split):
        df = df.set_index(['filename']).apply(pd.Series.explode).reset_index()
        output_name = output_name[:-4]+"_splitted.csv"
        print(output_name)
        print(df)
    
    df.to_csv(output_name,index=False,header=True, sep=",")
    
## use split = True if you want splitted output
## or False if you want merged output
json_to_csv(True)

['../dataset/raw\\OK TBS_S 2\\via_region_data_00016_00115.json', '../dataset/raw\\OK TBS_S 2\\via_region_data_00201_00401.json', '../dataset/raw\\OK TBS_S 2\\via_region_data_00401_00509.json', '../dataset/raw\\OK TBS_S 3\\20201007 via_region_data 1-110.json', '../dataset/raw\\OK TBS_S 3\\20201007 via_region_data 111-219.json', '../dataset/raw\\OK TBS_S 4\\20201012 via_region_data 0-120.json', '../dataset/raw\\OK TBS_S 4\\20201012 via_region_data 121-240.json', '../dataset/raw\\OK TBS_S 4\\20201012 via_region_data 241-360.json', '../dataset/raw\\OK TBS_S 4\\20201012 via_region_data 361-479.json', '../dataset/raw\\OK TDB-S 1\\20201002 via_region_data 0 -100.json', '../dataset/raw\\OK TDB-S 1\\20201002 via_region_data 101-480.json', '../dataset/raw\\TDB_M\\via_region_data_00001_00100.json', '../dataset/raw\\TDB_M\\via_region_data_00101_00200.json', '../dataset/raw\\TDB_M\\via_region_data_00201_00300.json', '../dataset/raw\\TDB_M\\via_region_data_00293_00400.json', '../dataset/raw\\TDB_M\\

In [112]:
## Clean Data
import pandas as pd

df = pd.read_csv("../dataset/processed/csv_converted_splitted.csv")
print("raw shape", df.shape)
print("raw columns", df.columns)

# delete plates (reduce the size from 6477 to 6167)
cups_glasses = df[df["class"] != "plate"]
cups_glasses = cups_glasses[cups_glasses["class"] != "platte"]
# print(cups_glasses[(cups_glasses["class"] == "vglass")])
cups_glasses.loc[(cups_glasses["class"] == "vglass"), ('class')] = "glass"

print("we still have nans", cups_glasses["class"].unique())
cups_glasses = cups_glasses.dropna()
print("only cups and glasses", cups_glasses.shape)
# size reduced from 6167 to 6117

acceptable_fills = ["0", "30", "100"]
cups_glasses_reduced = cups_glasses[cups_glasses["fill"].isin(acceptable_fills)]
print("only allowed fills", cups_glasses_reduced.shape)
# size reduced from 6117 to 5993

# write to a csv
cups_glasses_reduced.to_csv("../dataset/processed/reduced_cups_glasses.csv", index=False)

raw shape (15922, 7)
raw columns Index(['filename', 'class', 'fill', 'x', 'y', 'width', 'height'], dtype='object')
we still have nans ['glass' nan 'cup']
only cups and glasses (14778, 7)
only allowed fills (14646, 7)


In [119]:
# Split the dataset to Train and Test
# df = pd.read_csv("balanced_dataset.csv")
df = pd.read_csv("../dataset/processed/reduced_cups_glasses.csv")
print(df.shape)

fraction = 0.05
# Creating Test dataset
## Glass 0%
test = df[(df["fill"] == 0) & (df["class"] == "glass")].sample(frac=fraction)
## Glass 30%
test = test.append(df[(df["fill"] == 30) & (df["class"] == "glass")].sample(frac=fraction))
## Glass 100%
test = test.append(df[(df["fill"] == 100) & (df["class"] == "glass")].sample(frac=fraction))
## Cup 0%
test = test.append(df[(df["fill"] == 0) & (df["class"] == "cup")].sample(frac=fraction))
## Cup 30%
test = test.append(df[(df["fill"] == 30) & (df["class"] == "cup")].sample(frac=fraction))
## Cup 100%
test = test.append(df[(df["fill"] == 100) & (df["class"] == "cup")].sample(frac=fraction))
# print(df.loc[df['filename'].isin(test["filename"])])
# print(test["filename"])

## fixed the previous version which made the test data load all the bboxes on each test image.
## In the previous version,one test image contained only one bbox, 
## which made it possible for the training folder to have the same image as the test but only different bbox data.
## If you want to revert to the earlier algorithm you can change the fraction and comment a line of code below
test = test.append(df.loc[df['filename'].isin(test["filename"])]).drop_duplicates(keep=False)

# Create train dataset by excluding the test dataset
train = pd.concat([df, test, test]).drop_duplicates(keep=False)

print(test)
print(test.shape)
print(train)
print(train.shape)

train.to_csv("../dataset/processed/train_dataset.csv", index = False)
test.to_csv("../dataset/processed/test_dataset.csv", index = False)


(14646, 7)
                  filename  class  fill       x      y  width  height
1      OK_TBS_S_200016.jpg  glass     0  1140.0  731.0  167.0   186.0
30     OK_TBS_S_200031.jpg  glass   100   305.0  458.0  232.0   182.0
32     OK_TBS_S_200031.jpg  glass   100  1019.0  508.0  172.0   174.0
40     OK_TBS_S_200034.jpg  glass   100   380.0  847.0  160.0   169.0
41     OK_TBS_S_200034.jpg  glass   100   615.0  850.0  191.0   150.0
...                    ...    ...   ...     ...    ...    ...     ...
14640       TDB_M01999.jpg  glass    30   934.0  781.0  159.0   206.0
14641       TDB_M02000.jpg  glass    30   380.0  460.0  200.0   177.0
14642       TDB_M02000.jpg  glass    30   942.0  401.0  180.0   234.0
14643       TDB_M02000.jpg  glass    30   642.0  611.0  180.0   200.0
14645       TDB_M02000.jpg  glass    30   946.0  778.0  145.0   207.0

[2834 rows x 7 columns]
(2834, 7)
                  filename  class  fill       x      y  width  height
0      OK_TBS_S_200016.jpg  glass     0   24

In [120]:
import sys
sys.path.insert(0, 'D:\\Project\\AiTalents\\models\\')
sys.path.insert(0, 'D:\\Project\\AiTalents\\models\\research')
sys.path.insert(0, 'D:\\Project\\AiTalents\\models\\research\\object_detection')
print(sys.path)

['D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\', 'D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\', 'D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\', 'D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\', 'D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\', 'D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\', 'D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\', 'D:\\

In [121]:
## Functions to converting dataset to TFRecord
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd

import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple

def class_text_to_int(row_label):
    cls = {"glass_0":0, "glass_30":1, "glass_100":2, "cup_0":3, "cup_30":4, "cup_100":5}
    return cls[row_label]

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['x'] / width)
        xmaxs.append((row['x']+row['width']) / width)
        ymins.append(row['y'] / height)
        ymaxs.append((row['y']+row['height']) / height)
#         print(row["class"])
#         print(row["fill"])
        cls_fill = str(row['class'])+"_"+str(row['fill'])
        print(cls_fill)
        classes_text.append(cls_fill.encode('utf8'))
        classes.append(class_text_to_int(cls_fill))
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
    'image/height': dataset_util.int64_feature(height),
    'image/width': dataset_util.int64_feature(width),
    'image/filename': dataset_util.bytes_feature(filename),
    'image/source_id': dataset_util.bytes_feature(filename),
    'image/encoded': dataset_util.bytes_feature(encoded_jpg),
    'image/format': dataset_util.bytes_feature(image_format),
    'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
    'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
    'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
    'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
    'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
    'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def convert_df_to_tfrecord(images_path,input_df, output_path):
    with tf.io.TFRecordWriter(output_path) as writer:
        path = os.path.join(os.getcwd(), images_path)
        grouped = split(input_df, 'filename')
        for group in grouped:
            tf_example = create_tf_example(group, path)
            writer.write(tf_example.SerializeToString())

    print('Successfully created the TFRecords: {}'.format(output_path))

In [122]:
## Resize image
import cv2
from pathlib import Path
import numpy as np

def resize_image(size, input_path, output_path, x, y, width, height):
    img = cv2.imread(input_path)
    
    y_ = img.shape[0]
    x_ = img.shape[1]
    print(x_,y_)

    x_scale = size / x_
    y_scale = size / y_
    print(x_scale, y_scale)
    img = cv2.resize(img, (size,size), interpolation = cv2.INTER_AREA)
    print(img.shape)
    img = np.array(img);

    x = int(np.round(x * x_scale))
    y = int(np.round(y * y_scale))
    xmax = int(np.round(x+width * x_scale))
    ymax = int(np.round(y+height * y_scale))
    
#     img = cv2.rectangle(img, (int(x),int(y)), (int(xmax),int(ymax)), (255,0,255), 1)

    cv2.imwrite(output_path, img)
    return x,y,xmax,ymax

def split_dataset(input_path, output_path):
    if(not os.path.exists(output_path)):
        img = cv2.imread(input_path)
        cv2.imwrite(output_path, img)
    
df_train = pd.read_csv("../dataset/processed/train_dataset.csv")
df_test = pd.read_csv("../dataset/processed/test_dataset.csv")

dataset_path = "../dataset/processed/master_dataset/"
# output_path = "dataset/TDB_M_320/"
output_path = "../dataset/processed/master_splitted/"
output_train_path = output_path+"Train/"
output_test_path = output_path+"Test/"

path = Path(output_train_path)
path.mkdir(parents=True, exist_ok=True)

path = Path(output_test_path)
path.mkdir(parents=True, exist_ok=True)

## Resize The datasets to 320x320
for index, row in df_train.iterrows():
    print(row['filename'])
#     resize_image(320,dataset_path+row['filename'], output_train_path+row['filename'],row['x'],row['y'],row['width'],row['height'])
    split_dataset(dataset_path+row['filename'], output_train_path+row['filename'])

for index, row in df_test.iterrows():
    print(row['filename'])
#     resize_image(320,dataset_path+row['filename'], output_test_path+row['filename'],row['x'],row['y'],row['width'],row['height'])
    split_dataset(dataset_path+row['filename'], output_test_path+row['filename'])


OK_TBS_S_200016.jpg
OK_TBS_S_200017.jpg
OK_TBS_S_200017.jpg
OK_TBS_S_200018.jpg
OK_TBS_S_200018.jpg
OK_TBS_S_200019.jpg
OK_TBS_S_200019.jpg
OK_TBS_S_200020.jpg
OK_TBS_S_200020.jpg
OK_TBS_S_200021.jpg
OK_TBS_S_200021.jpg
OK_TBS_S_200022.jpg
OK_TBS_S_200022.jpg
OK_TBS_S_200023.jpg
OK_TBS_S_200023.jpg
OK_TBS_S_200024.jpg
OK_TBS_S_200024.jpg
OK_TBS_S_200025.jpg
OK_TBS_S_200025.jpg
OK_TBS_S_200026.jpg
OK_TBS_S_200026.jpg
OK_TBS_S_200027.jpg
OK_TBS_S_200027.jpg
OK_TBS_S_200028.jpg
OK_TBS_S_200028.jpg
OK_TBS_S_200029.jpg
OK_TBS_S_200029.jpg
OK_TBS_S_200030.jpg
OK_TBS_S_200030.jpg
OK_TBS_S_200031.jpg
OK_TBS_S_200032.jpg
OK_TBS_S_200032.jpg
OK_TBS_S_200032.jpg
OK_TBS_S_200032.jpg
OK_TBS_S_200033.jpg
OK_TBS_S_200033.jpg
OK_TBS_S_200033.jpg
OK_TBS_S_200034.jpg
OK_TBS_S_200035.jpg
OK_TBS_S_200036.jpg
OK_TBS_S_200036.jpg
OK_TBS_S_200036.jpg
OK_TBS_S_200037.jpg
OK_TBS_S_200037.jpg
OK_TBS_S_200037.jpg
OK_TBS_S_200038.jpg
OK_TBS_S_200038.jpg
OK_TBS_S_200038.jpg
OK_TBS_S_200039.jpg
OK_TBS_S_200040.jpg


OK_TBS_S_200487.jpg
OK_TBS_S_200487.jpg
OK_TBS_S_200487.jpg
OK_TBS_S_200488.jpg
OK_TBS_S_200489.jpg
OK_TBS_S_200489.jpg
OK_TBS_S_200489.jpg
OK_TBS_S_200489.jpg
OK_TBS_S_200489.jpg
OK_TBS_S_200489.jpg
OK_TBS_S_200490.jpg
OK_TBS_S_200490.jpg
OK_TBS_S_200490.jpg
OK_TBS_S_200490.jpg
OK_TBS_S_200491.jpg
OK_TBS_S_200491.jpg
OK_TBS_S_200491.jpg
OK_TBS_S_200491.jpg
OK_TBS_S_200491.jpg
OK_TBS_S_200492.jpg
OK_TBS_S_200492.jpg
OK_TBS_S_200492.jpg
OK_TBS_S_200492.jpg
OK_TBS_S_200492.jpg
OK_TBS_S_200492.jpg
OK_TBS_S_200493.jpg
OK_TBS_S_200493.jpg
OK_TBS_S_200493.jpg
OK_TBS_S_200493.jpg
OK_TBS_S_200493.jpg
OK_TBS_S_200494.jpg
OK_TBS_S_200495.jpg
OK_TBS_S_200495.jpg
OK_TBS_S_200495.jpg
OK_TBS_S_200495.jpg
OK_TBS_S_200495.jpg
OK_TBS_S_200495.jpg
OK_TBS_S_200496.jpg
OK_TBS_S_200496.jpg
OK_TBS_S_200496.jpg
OK_TBS_S_200496.jpg
OK_TBS_S_200496.jpg
OK_TBS_S_200497.jpg
OK_TBS_S_200497.jpg
OK_TBS_S_200497.jpg
OK_TBS_S_200497.jpg
OK_TBS_S_200497.jpg
OK_TBS_S_200498.jpg
OK_TBS_S_200498.jpg
OK_TBS_S_200498.jpg


OK_TBS_S_3Unknown-117.jpg
OK_TBS_S_3Unknown-117.jpg
OK_TBS_S_3Unknown-117.jpg
OK_TBS_S_3Unknown-118.jpg
OK_TBS_S_3Unknown-118.jpg
OK_TBS_S_3Unknown-118.jpg
OK_TBS_S_3Unknown-118.jpg
OK_TBS_S_3Unknown-118.jpg
OK_TBS_S_3Unknown-118.jpg
OK_TBS_S_3Unknown-118.jpg
OK_TBS_S_3Unknown-118.jpg
OK_TBS_S_3Unknown-118.jpg
OK_TBS_S_3Unknown-118.jpg
OK_TBS_S_3Unknown-119.jpg
OK_TBS_S_3Unknown-119.jpg
OK_TBS_S_3Unknown-119.jpg
OK_TBS_S_3Unknown-119.jpg
OK_TBS_S_3Unknown-119.jpg
OK_TBS_S_3Unknown-119.jpg
OK_TBS_S_3Unknown-119.jpg
OK_TBS_S_3Unknown-119.jpg
OK_TBS_S_3Unknown-119.jpg
OK_TBS_S_3Unknown-119.jpg
OK_TBS_S_3Unknown-120.jpg
OK_TBS_S_3Unknown-120.jpg
OK_TBS_S_3Unknown-121.jpg
OK_TBS_S_3Unknown-122.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Unknown-123.jpg
OK_TBS_S_3Un

OK_TBS_S_4Unknown-39.jpg
OK_TBS_S_4Unknown-39.jpg
OK_TBS_S_4Unknown-39.jpg
OK_TBS_S_4Unknown-39.jpg
OK_TBS_S_4Unknown-39.jpg
OK_TBS_S_4Unknown-40.jpg
OK_TBS_S_4Unknown-40.jpg
OK_TBS_S_4Unknown-40.jpg
OK_TBS_S_4Unknown-40.jpg
OK_TBS_S_4Unknown-40.jpg
OK_TBS_S_4Unknown-41.jpg
OK_TBS_S_4Unknown-42.jpg
OK_TBS_S_4Unknown-42.jpg
OK_TBS_S_4Unknown-42.jpg
OK_TBS_S_4Unknown-42.jpg
OK_TBS_S_4Unknown-42.jpg
OK_TBS_S_4Unknown-43.jpg
OK_TBS_S_4Unknown-43.jpg
OK_TBS_S_4Unknown-43.jpg
OK_TBS_S_4Unknown-43.jpg
OK_TBS_S_4Unknown-43.jpg
OK_TBS_S_4Unknown-44.jpg
OK_TBS_S_4Unknown-44.jpg
OK_TBS_S_4Unknown-44.jpg
OK_TBS_S_4Unknown-44.jpg
OK_TBS_S_4Unknown-44.jpg
OK_TBS_S_4Unknown-45.jpg
OK_TBS_S_4Unknown-45.jpg
OK_TBS_S_4Unknown-45.jpg
OK_TBS_S_4Unknown-45.jpg
OK_TBS_S_4Unknown-45.jpg
OK_TBS_S_4Unknown-46.jpg
OK_TBS_S_4Unknown-46.jpg
OK_TBS_S_4Unknown-46.jpg
OK_TBS_S_4Unknown-46.jpg
OK_TBS_S_4Unknown-46.jpg
OK_TBS_S_4Unknown-47.jpg
OK_TBS_S_4Unknown-47.jpg
OK_TBS_S_4Unknown-47.jpg
OK_TBS_S_4Unknown-47.jpg


OK_TBS_S_4Unknown-257.jpg
OK_TBS_S_4Unknown-257.jpg
OK_TBS_S_4Unknown-257.jpg
OK_TBS_S_4Unknown-257.jpg
OK_TBS_S_4Unknown-257.jpg
OK_TBS_S_4Unknown-258.jpg
OK_TBS_S_4Unknown-258.jpg
OK_TBS_S_4Unknown-258.jpg
OK_TBS_S_4Unknown-258.jpg
OK_TBS_S_4Unknown-258.jpg
OK_TBS_S_4Unknown-259.jpg
OK_TBS_S_4Unknown-260.jpg
OK_TBS_S_4Unknown-260.jpg
OK_TBS_S_4Unknown-260.jpg
OK_TBS_S_4Unknown-260.jpg
OK_TBS_S_4Unknown-260.jpg
OK_TBS_S_4Unknown-261.jpg
OK_TBS_S_4Unknown-262.jpg
OK_TBS_S_4Unknown-262.jpg
OK_TBS_S_4Unknown-262.jpg
OK_TBS_S_4Unknown-262.jpg
OK_TBS_S_4Unknown-262.jpg
OK_TBS_S_4Unknown-263.jpg
OK_TBS_S_4Unknown-263.jpg
OK_TBS_S_4Unknown-263.jpg
OK_TBS_S_4Unknown-263.jpg
OK_TBS_S_4Unknown-263.jpg
OK_TBS_S_4Unknown-264.jpg
OK_TBS_S_4Unknown-264.jpg
OK_TBS_S_4Unknown-264.jpg
OK_TBS_S_4Unknown-264.jpg
OK_TBS_S_4Unknown-264.jpg
OK_TBS_S_4Unknown-265.jpg
OK_TBS_S_4Unknown-265.jpg
OK_TBS_S_4Unknown-265.jpg
OK_TBS_S_4Unknown-265.jpg
OK_TBS_S_4Unknown-265.jpg
OK_TBS_S_4Unknown-266.jpg
OK_TBS_S_4Un

...

OK_TBS_S_3Unknown-143.jpg
OK_TBS_S_3Unknown-143.jpg
OK_TBS_S_3Unknown-143.jpg
OK_TBS_S_3Unknown-143.jpg
OK_TBS_S_3Unknown-143.jpg
OK_TBS_S_3Unknown-143.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-144.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-146.jpg
OK_TBS_S_3Unknown-149.jpg
OK_TBS_S_3Unknown-149.jpg
OK_TBS_S_3Unknown-149.jpg
OK_TBS_S_3Unknown-149.jpg
OK_TBS_S_3Unknown-149.jpg
OK_TBS_S_3Un

OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-199.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-203.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Unknown-206.jpg
OK_TBS_S_3Un

OK_TBS_S_4Unknown-269.jpg
OK_TBS_S_4Unknown-275.jpg
OK_TBS_S_4Unknown-275.jpg
OK_TBS_S_4Unknown-275.jpg
OK_TBS_S_4Unknown-275.jpg
OK_TBS_S_4Unknown-288.jpg
OK_TBS_S_4Unknown-288.jpg
OK_TBS_S_4Unknown-288.jpg
OK_TBS_S_4Unknown-288.jpg
OK_TBS_S_4Unknown-289.jpg
OK_TBS_S_4Unknown-289.jpg
OK_TBS_S_4Unknown-289.jpg
OK_TBS_S_4Unknown-289.jpg
OK_TBS_S_4Unknown-290.jpg
OK_TBS_S_4Unknown-290.jpg
OK_TBS_S_4Unknown-290.jpg
OK_TBS_S_4Unknown-290.jpg
OK_TBS_S_4Unknown-293.jpg
OK_TBS_S_4Unknown-293.jpg
OK_TBS_S_4Unknown-293.jpg
OK_TBS_S_4Unknown-293.jpg
OK_TBS_S_4Unknown-294.jpg
OK_TBS_S_4Unknown-294.jpg
OK_TBS_S_4Unknown-294.jpg
OK_TBS_S_4Unknown-294.jpg
OK_TBS_S_4Unknown-295.jpg
OK_TBS_S_4Unknown-295.jpg
OK_TBS_S_4Unknown-295.jpg
OK_TBS_S_4Unknown-295.jpg
OK_TBS_S_4Unknown-297.jpg
OK_TBS_S_4Unknown-297.jpg
OK_TBS_S_4Unknown-297.jpg
OK_TBS_S_4Unknown-297.jpg
OK_TBS_S_4Unknown-310.jpg
OK_TBS_S_4Unknown-310.jpg
OK_TBS_S_4Unknown-310.jpg
OK_TBS_S_4Unknown-310.jpg
OK_TBS_S_4Unknown-312.jpg
OK_TBS_S_4Un

OK_TDB-S_1Unknown-192.jpg
OK_TDB-S_1Unknown-201.jpg
OK_TDB-S_1Unknown-201.jpg
OK_TDB-S_1Unknown-201.jpg
OK_TDB-S_1Unknown-201.jpg
OK_TDB-S_1Unknown-204.jpg
OK_TDB-S_1Unknown-204.jpg
OK_TDB-S_1Unknown-204.jpg
OK_TDB-S_1Unknown-204.jpg
OK_TDB-S_1Unknown-207.jpg
OK_TDB-S_1Unknown-207.jpg
OK_TDB-S_1Unknown-207.jpg
OK_TDB-S_1Unknown-207.jpg
OK_TDB-S_1Unknown-208.jpg
OK_TDB-S_1Unknown-208.jpg
OK_TDB-S_1Unknown-208.jpg
OK_TDB-S_1Unknown-208.jpg
OK_TDB-S_1Unknown-211.jpg
OK_TDB-S_1Unknown-211.jpg
OK_TDB-S_1Unknown-211.jpg
OK_TDB-S_1Unknown-211.jpg
OK_TDB-S_1Unknown-212.jpg
OK_TDB-S_1Unknown-212.jpg
OK_TDB-S_1Unknown-212.jpg
OK_TDB-S_1Unknown-212.jpg
OK_TDB-S_1Unknown-217.jpg
OK_TDB-S_1Unknown-217.jpg
OK_TDB-S_1Unknown-217.jpg
OK_TDB-S_1Unknown-217.jpg
OK_TDB-S_1Unknown-233.jpg
OK_TDB-S_1Unknown-233.jpg
OK_TDB-S_1Unknown-233.jpg
OK_TDB-S_1Unknown-233.jpg
OK_TDB-S_1Unknown-239.jpg
OK_TDB-S_1Unknown-239.jpg
OK_TDB-S_1Unknown-239.jpg
OK_TDB-S_1Unknown-239.jpg
OK_TDB-S_1Unknown-245.jpg
OK_TDB-S_1Un

TDB_M00633.jpg
TDB_M00649.jpg
TDB_M00649.jpg
TDB_M00649.jpg
TDB_M00651.jpg
TDB_M00651.jpg
TDB_M00651.jpg
TDB_M00651.jpg
TDB_M00653.jpg
TDB_M00653.jpg
TDB_M00660.jpg
TDB_M00660.jpg
TDB_M00662.jpg
TDB_M00662.jpg
TDB_M00676.jpg
TDB_M00676.jpg
TDB_M00678.jpg
TDB_M00678.jpg
TDB_M00679.jpg
TDB_M00679.jpg
TDB_M00685.jpg
TDB_M00685.jpg
TDB_M00693.jpg
TDB_M00693.jpg
TDB_M00703.jpg
TDB_M00703.jpg
TDB_M00712.jpg
TDB_M00712.jpg
TDB_M00717.jpg
TDB_M00717.jpg
TDB_M00722.jpg
TDB_M00722.jpg
TDB_M00737.jpg
TDB_M00737.jpg
TDB_M00740.jpg
TDB_M00740.jpg
TDB_M00740.jpg
TDB_M00740.jpg
TDB_M00741.jpg
TDB_M00741.jpg
TDB_M00741.jpg
TDB_M00741.jpg
TDB_M00743.jpg
TDB_M00743.jpg
TDB_M00743.jpg
TDB_M00743.jpg
TDB_M00749.jpg
TDB_M00749.jpg
TDB_M00749.jpg
TDB_M00749.jpg
TDB_M00761.jpg
TDB_M00761.jpg
TDB_M00761.jpg
TDB_M00761.jpg
TDB_M00765.jpg
TDB_M00765.jpg
TDB_M00765.jpg
TDB_M00773.jpg
TDB_M00773.jpg
TDB_M00773.jpg
TDB_M00775.jpg
TDB_M00775.jpg
TDB_M00775.jpg
TDB_M00775.jpg
TDB_M00778.jpg
TDB_M00778.jpg
TDB_M00778

In [123]:
# Create TFRecord
convert_df_to_tfrecord(output_train_path,df_train, "../dataset/processed/train.tfrecord")
convert_df_to_tfrecord(output_test_path,df_test, "../dataset/processed/test.tfrecord")

glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_0
glass_100
glass_100
glass_100
glass_100
glass_100
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glas

glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_0
glass_0
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
gl

glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0


glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_0
glass_0
glass_0
glass_0
glass_0
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_0
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_0
glass_0
glass_0
glass_0
glass_0
glass_30
glass_30
gla

glass_30
glass_30
glass_30
glass_30
glass_30
glass_0
glass_0
glass_0
glass_0
glass_0
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_0
glass_0
glass_0
glass_0
glass_0
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_0
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
gla

...

glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
g

glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
g

glass_0
glass_0
glass_0
glass_0
glass_0
cup_100
cup_30
cup_30
cup_30
cup_30
cup_0
cup_0
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
cup_100
glass_0
glass_0
cup_100
glass_0
cup_100
glass_0
glass_0
glass_0
cup_30
cup_30
glass_0
cup_30
glass_0
cup_30
glass_0
glass_0
cup_30
glass_0
glass_0
glass_0
cup_30
cup_30
glass_0
glass_0
glass_0
cup_30
cup_30
glass_0
glass_0
glass_0
glass_0
glass_0
cup_30
glass_0
glass_0
cup_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
cup_0
glass_0
cup_0
glass_0
glass_0
cup_0
glass_0
glass_0
cup_0
cup_0
cup_0
glass_0
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
glass_100
gla

glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30


glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_30
glass_30
glass_30
glass_0
glass_0
glass_0
glass_30
glass_30
glass_0
glass_0
glass_30
glass_30
glass_0
glass_30
glass_30
glass_30
glass_0
glass_0
glass_30
glass_30
glass_0
cup_0
cup_0
glass_0
glass_0
cup_0
glass_0
cup_0
cup_0
glass_0
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
glass_0
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
cup_30
glass_30
cup_30
cup_0
glass_0
cup_0
cup_30
cup_30
glass_0
cup_0
glass_0
cup_0
cup_0
cup_0
glass_0
glass_0
glass_0
glass_0
glass_0
cup_0
cup_0
cup_0
cup_0
cup_0
cup_0
cup_0
cup_0
cup_0
cup_30
cup_0
cu

In [124]:
# Create Class Pbtxt
classes = ("glass_0", "glass_30", "glass_100", "cup_0", "cup_30", "cup_100")
def create_class_pbtxt(class_names, output_path):
    end = '\n'
    s = ' '
    for idx, cls in enumerate(class_names):
        out = ''
        out += 'item' + s + '{' + end
        out += s*2 + 'name:' + ' ' + '\"' + cls + '\"' + end
        out += s*2 + 'id:' + ' ' + (str(idx+1)) + end
        out += s*2 + 'display_name:' + ' ' + '\"' + cls + '\"' + end
        out += '}' + end
        with open(output_path, 'a') as f:
            f.write(out)
    print("pbtx file created successfully")
        
create_class_pbtxt(classes,'../dataset/processed/class.pbtxt')

pbtx file created successfully


In [ ]:
## What to do next? (on SSD Train)
# Train
# Test
# Convert Model to Tensorflow JS